In [1]:
import os

dataDir = '/home/wzheng/globus_dest/xzhang_mar/'

listFiles = os.listdir(dataDir)
listFiles

['Fe9Cr_61116_stage1_ff_000120.edf.ge5',
 'Fe9Cr_61116_stage1_ff_000118.edf.ge5',
 'dark_before_000128.edf.ge5',
 'dark_before_000117.edf.ge5',
 'dark_before_000023.edf.ge5',
 'dark_before_000137.edf.ge5',
 'Fe9Cr_61116_stage1_ff_000119.edf.ge5',
 'dark_before_000122.edf.ge5',
 'dark_before_000010.edf.ge5']

In [2]:
darkFilesList = []
darkFilesDict = {}

for fileString in listFiles:
    if fileString.startswith('dark_before'):
        # find the index of each dark before
        idx = int(fileString[12:18])
        #print(int(fileString[12:18]))
        darkFilesDict[idx] = fileString

darkFilesDict

{128: 'dark_before_000128.edf.ge5',
 117: 'dark_before_000117.edf.ge5',
 23: 'dark_before_000023.edf.ge5',
 137: 'dark_before_000137.edf.ge5',
 122: 'dark_before_000122.edf.ge5',
 10: 'dark_before_000010.edf.ge5'}

In [3]:
darkFilesKeys = sorted(darkFilesDict)
darkFilesKeys

[10, 23, 117, 122, 128, 137]

In [4]:
fe9crFilesList = []
fe9crFilesDict4Idx = {}

# park_ss_ff

for fileString in listFiles:
    if fileString.startswith('Fe9Cr_61116'):
        # find the index of each file
        x = fileString.split("_")
        idx = int(x[-1].split(".")[0])
        fe9crFilesDict4Idx[fileString] = idx
#         idx = int(fileString[12:18])
#         #print(int(fileString[12:18]))
#         darkFilesDict[idx] = fileString
#         darkFiles.append(fileString)

# 
fe9crFilesDict4Idx 

{'Fe9Cr_61116_stage1_ff_000120.edf.ge5': 120,
 'Fe9Cr_61116_stage1_ff_000118.edf.ge5': 118,
 'Fe9Cr_61116_stage1_ff_000119.edf.ge5': 119}

In [5]:
# next is to find the dark files for each file
darkFilesDict4Idx = {}

# park_ss_ff

for fileString in listFiles:
    if fileString.startswith('Fe9Cr_61116'):
        # find the index of each file
        x = fileString.split("_")
        idx = int(x[-1].split(".")[0])
        # next is to find the corresponding dark file
        for darkKey in darkFilesKeys:
            if darkKey >= idx:
                print(str(idx) + "-" + str(darkKey))
                darkFilesDict4Idx[fileString] = darkFilesDict[darkKey]
                break

darkFilesDict4Idx

120-122
118-122
119-122


{'Fe9Cr_61116_stage1_ff_000120.edf.ge5': 'dark_before_000122.edf.ge5',
 'Fe9Cr_61116_stage1_ff_000118.edf.ge5': 'dark_before_000122.edf.ge5',
 'Fe9Cr_61116_stage1_ff_000119.edf.ge5': 'dark_before_000122.edf.ge5'}

In [6]:
len(darkFilesDict4Idx)

3

In [7]:
# now start to convert ge5 files to patches
from src.datasets.ge2patch import *

In [8]:
# need to create the following dir first
outDir = '/home/wzheng/fe9cr_patches/'

i = 1

for key in darkFilesDict4Idx:
    print(f"Reading dark file from {dataDir+darkFilesDict4Idx[key]} ... ")
    dark = ge_raw2array(dataDir+darkFilesDict4Idx[key], skip_frm=0).mean(axis=0).astype(np.float32)
    print(f"Done with reading dark file from {dataDir+darkFilesDict4Idx[key]} with {dark}")
    print(f"The input file is {key}")
    outFile = outDir + key[:-3] + "h5"
    print(f"The output file is {outFile}")
    print(f"Start to generate patches for file {key} ({i}/{len(darkFilesDict4Idx)})...")
    ge_raw2patch(gefname=dataDir+key, ofn=outFile, dark=dark, 
                 bkgd=100, psz=15, skip_frm=0, min_intensity=0, max_r=None)
    print(f"Done generating patches for file {key} ({i}/{len(darkFilesDict4Idx)})...")
    i += 1

Reading dark file from /home/wzheng/globus_dest/xzhang_mar/dark_before_000122.edf.ge5 ... 
Done with reading dark file from /home/wzheng/globus_dest/xzhang_mar/dark_before_000122.edf.ge5 with [[1650.2 1658.5 1658.7 ... 1835.7 1833.8 1831.1]
 [1668.4 1675.  1672.4 ... 1826.1 1824.9 1823.4]
 [1678.1 1682.2 1680.4 ... 1833.8 1834.3 1831.8]
 ...
 [1679.4 1680.4 1682.1 ... 1736.8 1734.6 1735.2]
 [1672.7 1671.2 1671.9 ... 1731.4 1730.9 1727.9]
 [1637.1 1636.  1633.3 ... 1701.4 1700.7 1699.7]]
The input file is Fe9Cr_61116_stage1_ff_000120.edf.ge5
The output file is /home/wzheng/fe9cr_patches/Fe9Cr_61116_stage1_ff_000120.edf.h5
Start to generate patches for file Fe9Cr_61116_stage1_ff_000120.edf.ge5 (1/3)...
data in the file are not completely parsed, 6545408 left over
66480 patches of size 15 cropped from /home/wzheng/globus_dest/xzhang_mar/Fe9Cr_61116_stage1_ff_000120.edf.ge5, 70580 are too big.
Done generating patches for file Fe9Cr_61116_stage1_ff_000120.edf.ge5 (1/3)...
Reading dark file 